In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

cvs_health = pd.read_csv('CVD_cleaned.csv')
print(cvs_health)

       General_Health                  Checkup Exercise Heart_Disease  \
0                Poor  Within the past 2 years       No            No   
1           Very Good     Within the past year       No           Yes   
2           Very Good     Within the past year      Yes            No   
3                Poor     Within the past year      Yes           Yes   
4                Good     Within the past year       No            No   
...               ...                      ...      ...           ...   
308849      Very Good     Within the past year      Yes            No   
308850           Fair  Within the past 5 years      Yes            No   
308851      Very Good      5 or more years ago      Yes            No   
308852      Very Good     Within the past year      Yes            No   
308853      Excellent     Within the past year      Yes            No   

       Skin_Cancer Other_Cancer Depression  \
0               No           No         No   
1               No           No

In [3]:
cvs_health.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0


In [2]:
#data discretation 

# loop, 15% dos features (10-20) - usar média ou mediana para missing values, ou 0
# low, medium, high - pegar num atributo dividi-lo por Bins



#data normalization 



In [12]:
import pandas as pd

# create a sample dataframe
df = pd.DataFrame(cvs_health)

# definir limites dos bins
bins = [0, 40, 80, 120]
labels = ['low', 'medium', 'high']

df['Fruit_Consumption'] = pd.cut(df['Fruit_Consumption'], bins, labels=labels)

#df['age_bin'] = pd.qcut(df['age'], q=4, labels=False)

print(df['Fruit_Consumption'] )
df.to_csv('file_teste.csv', index=False) # salva os bins criados para fruit consumption 
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=['Unnamed: 0'])
    df = df.to_csv('file_teste.csv') 

0         low
1         low
2         low
3         low
4         low
         ... 
308849    low
308850    low
308851    low
308852    low
308853    low
Name: Fruit_Consumption, Length: 308854, dtype: category
Categories (3, object): ['low' < 'medium' < 'high']


In [11]:
# health = pd.read_csv('file_teste.csv')
# df = pd.DataFrame(health)

# # definir limites dos bins
# bins = [0, 40, 80, 130]
# labels = ['low', 'medium', 'high']

# df['Green_Vegetables_Consumption'] = pd.cut(df['Green_Vegetables_Consumption'], bins, labels=labels)

# #df['age_bin'] = pd.qcut(df['age'], q=4, labels=False)

# df.to_csv('file_teste.csv', index=False)

# print(df['Green_Vegetables_Consumption'].value_counts())



Green_Vegetables_Consumption
low       275583
medium      9079
high        2803
Name: count, dtype: int64


In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Carregar o dataset
file_path = 'CVD_cleaned.csv'  # Substitua pelo caminho do ficheiro
data = pd.read_csv(file_path)

# Copiar o dataset original para evitar alterações diretas
data_normalized = data.copy()

# Identificar as colunas categóricas
categorical_columns = [
    "General_Health",
    "Checkup",
    "Exercise",
    "Heart_Disease",
    "Skin_Cancer",
    "Other_Cancer",
    "Depression",
    "Diabetes",
    "Arthritis",
    "Sex",
    "Age_Category",
    "Smoking_History",
]

# Identificar as colunas numéricas contínuas
numeric_columns = [
    "Height_(cm)",
    "Weight_(kg)",
    "BMI",
    "Alcohol_Consumption",
    "Fruit_Consumption",
    "Green_Vegetables_Consumption",
    "FriedPotato_Consumption",
]

# Codificar as colunas categóricas
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data_normalized[col] = le.fit_transform(data_normalized[col])
    label_encoders[col] = le

# Normalizar apenas as colunas numéricas contínuas
scaler = MinMaxScaler()
data_normalized[numeric_columns] = scaler.fit_transform(data_normalized[numeric_columns])

# Salvar o dataset normalizado em um novo arquivo
output_file_path = 'CVD_normalized_corrected.csv'  # Nome do arquivo de saída
data_normalized.to_csv(output_file_path, index=False)

print(f"Dataset normalizado salvo como: {output_file_path}")


Dataset normalizado salvo como: CVD_normalized_corrected.csv


In [6]:
data_normalized.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,3,2,0,0,0,0,0,0,1,0,10,0.393333,0.028761,0.028863,1,0.000000,0.250000,0.125000,0.09375
1,4,4,0,1,0,0,0,2,0,0,10,0.493333,0.194576,0.186347,0,0.000000,0.250000,0.000000,0.03125
2,4,4,1,0,0,0,0,2,0,0,8,0.480000,0.236878,0.245676,0,0.133333,0.100000,0.023438,0.12500
3,3,4,1,1,0,0,0,2,0,1,11,0.593333,0.255493,0.191387,0,0.000000,0.250000,0.234375,0.06250
4,2,4,0,0,0,0,0,0,0,1,12,0.666667,0.236878,0.141450,1,0.000000,0.066667,0.031250,0.00000


In [ ]:
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer

# Carregar o dataset
file_path = 'CVD_cleaned.csv'  # chamar file com data reduction 
data = pd.read_csv(file_path)

data_discretized = data.copy()

# colunas numéricas contínuas
numeric_columns = [
    "Height_(cm)",
    "Weight_(kg)",
    "BMI",
    "Alcohol_Consumption",
    "Fruit_Consumption",
    "Green_Vegetables_Consumption",
    "FriedPotato_Consumption",
]

# Configurar o discretizador
n_bins = 10  # Número de bins
discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='kmeans')

# Aplicar a discretização nas colunas numéricas
data_discretized[numeric_columns] = discretizer.fit_transform(data_discretized[numeric_columns])

# Mostrar os limites dos bins para cada coluna
for col in numeric_columns:
    print(f"Intervalos para {col}: {discretizer.bin_edges_}")

# Salvar o dataset discretizado em um novo arquivo
output_file_path = 'CVD_discretized.csv'  # Nome do arquivo de saída
data_discretized.to_csv(output_file_path, index=False)

print(f"Dataset discretizado salvo como: {output_file_path}")


Intervalos para Height_(cm): [array([ 91.        , 113.08191851, 134.9314907 , 149.87360995,
        158.85113528, 167.04169096, 176.11752951, 185.37024537,
        193.80998322, 206.01452399, 241.        ])
 array([ 25.4       ,  64.74706478,  79.28908533,  95.57356651,
        114.6411917 , 136.25410898, 159.30928047, 182.96622745,
        210.01475959, 247.5831875 , 293.02      ])
 array([12.11      , 22.50535001, 26.05568608, 29.29676517, 32.53679495,
        36.09452272, 40.3344357 , 45.7375068 , 53.36228322, 66.68853041,
        98.44      ])
 array([ 0.        ,  2.15290448,  5.73651157,  8.74490854, 10.98859863,
        13.66041797, 17.64705289, 21.86609581, 24.47280149, 27.25770973,
        30.        ])
 array([  0.        ,  12.37929827,  23.70149145,  34.74663647,
         43.50964749,  53.62425513,  69.60223867,  84.60528045,
         94.39327328, 109.3915292 , 120.        ])
 array([  0.        ,   9.51694513,  22.0972075 ,  35.10114164,
         50.36767057,  67.48360229

In [13]:
data_discretized.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,3.0,0.0,0.0,Yes,0.0,2.0,1.0,1.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,4.0,1.0,2.0,No,0.0,2.0,0.0,0.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,4.0,2.0,4.0,No,1.0,0.0,0.0,1.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,6.0,2.0,2.0,No,0.0,2.0,2.0,0.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,7.0,2.0,1.0,Yes,0.0,0.0,0.0,0.0
